In [151]:
from functools import lru_cache

def tab(func):
    var = {
        'level':-1,
        'count':0
    }
    
    def wrapper(*args, **kw):
        var['level'] += 1
        print('-' * var['level'], '空' if args[1] == '' else args[1], '空' if args[2] == '' else args[2])
        r = func(*args, **kw)
        var['count'] += 1
        var['level'] -= 1
        return r
    
    
    return wrapper



class Solution:
    
    @lru_cache()
    @tab
    def isMatch(self, s: str, p: str) -> bool:
        
        if p == "" and s != "":
            return False
        if p == "" and s == "":
            return True
        
        # p != ""
        if p[-1] == "*":
            if len(p) == 1:
                return False
            # 重复0次
            # 重复1次及以上
            return self.isMatch(s, p[:-2]) or self.isMatch(s, p+p[-2])
        # p != "" and p[-1] != "*"
        
        if p[-1] == ".":
            if s == "":
                return False
            return self.isMatch(s[:-1], p[:-1])  # s变短，p变短
        
        # p != "" and p[-1] != "*" and p != "." 
        if s != "" and p[-1] == s[-1]:
            return self.isMatch(s[:-1], p[:-1])  # s变短，p变短
        else:
            # s == ""
            # 但已排除p != ".*"的可能
            # s != ""
            # p[-1] != s[-1]
            return False


SyntaxError: no binding for nonlocal 'var' found (<ipython-input-151-1349ddd24db8>, line 4)

In [152]:
solu = Solution()

s = "mississippi"
p = "mis*is*p*."
print(solu.isMatch(s, p) == False)

s = "aa"
p = "a"
print(solu.isMatch(s, p))

s = "ab"
p = ".*"
print(solu.isMatch(s, p))

s = "aab"
p = "c*a*b"
print(solu.isMatch(s, p))

s = "mississippi"
p = "mis*is*p*."
print(solu.isMatch(s, p))

s = "a"
p = ".*..a*"
print(solu.isMatch(s, p))

 mississippi mis*is*p*.
- mississipp mis*is*p*
-- mississipp mis*is*
--- mississipp mis*i
--- mississipp mis*is*s
-- mississipp mis*is*p*p
--- mississip mis*is*p*
---- mississip mis*is*
----- mississip mis*i
----- mississip mis*is*s
---- mississip mis*is*p*p
----- mississi mis*is*p*
------ mississi mis*is*
------- mississi mis*i
-------- mississ mis*
--------- mississ mi
--------- mississ mis*s
---------- missis mis*
----------- missis mi
----------- missis mis*s
------------ missi mis*
------------- missi mi
-------------- miss m
------------- missi mis*s
------- mississi mis*is*s
------ mississi mis*is*p*p
True
 aa a
- a 空
False
 ab .*
- ab 空
- ab .*.
-- a .*
--- a .*.
---- 空 .*
----- 空 空
True
 aab c*a*b
- aa c*a*
-- aa c*
--- aa 空
--- aa c*c
-- aa c*a*a
--- a c*a*
---- a c*
----- a c*c
---- a c*a*a
----- 空 c*a*
------ 空 c*
True
False
 a .*..a*
- a .*..
-- 空 .*.
- a .*..a*a
-- 空 .*..a*
--- 空 .*..
--- 空 .*..a*a
False


In [155]:
Solution.isMatch.__dict__


{'__module__': '__main__',
 '__name__': 'wrapper',
 '__qualname__': 'tab.<locals>.wrapper',
 '__doc__': None,
 '__annotations__': {},
 '__wrapped__': <function __main__.tab.<locals>.wrapper(*args, **kw)>}

In [4]:
class Solution(object):
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """

        
        '''
        状态转移表的构建 ↓
        '''
        dic = {}
        state = 1
        for idx, i in enumerate(p):
            if 'a' <= i <= "z" or i == '.':
                dic.setdefault(state, {}).update({i: state + 1})
                state += 1
            elif i == '*':
                dic[state - 1][p[idx - 1]] = state - 1
                dic[state - 1]['$'] = state
        print(dic)

        '''
        dfs部分
        '''
        def helper(sta, px):
            if px:
                try:
                    tmp1 = dic[sta].get(px[0], False)
                    tmp2 = dic[sta].get('.', False)
                    tmp3 = dic[sta].get('$', False)
                except:
                    return False
                if tmp1:
                    if helper(tmp1, px[1:]):
                        return True
                if tmp2:
                    if helper(tmp2, px[1:]):
                        return True
                if tmp3:
                    if helper(tmp3,px):
                        return True
                return False
            else:
                try:
                    tmp = dic[sta].get('$', False)
                    return helper(tmp,px)
                except:
                    pass
                if sta == state:
                    return True
                return False
        return helper(1, s)


s = Solution()
tests = [
    ("babbcaacbccbbbbbabb","bb*.b*b*a*aba*c"),
    ("aa","a*"),
    ("abbabaaaaaaacaa", "a*.*b.a.*c*b*a*c*"),
    ("aaba", "ab*a*c*a"),
    ("aaa", "a*a"),
    ("abcdede", "ab.*de"),
]
for item in tests:
    res = s.isMatch(*item)
    print(res)


{1: {'b': 2}, 2: {'b': 2, '$': 3}, 3: {'.': 4}, 4: {'b': 4, '$': 5}, 5: {'b': 5, '$': 6}, 6: {'a': 6, '$': 7}, 7: {'a': 8}, 8: {'b': 9}, 9: {'a': 9, '$': 10}, 10: {'c': 11}}
False
{1: {'a': 1, '$': 2}}
True
{1: {'a': 1, '$': 2}, 2: {'.': 2, '$': 3}, 3: {'b': 4}, 4: {'.': 5}, 5: {'a': 6}, 6: {'.': 6, '$': 7}, 7: {'c': 7, '$': 8}, 8: {'b': 8, '$': 9}, 9: {'a': 9, '$': 10}, 10: {'c': 10, '$': 11}}
True
{1: {'a': 2}, 2: {'b': 2, '$': 3}, 3: {'a': 3, '$': 4}, 4: {'c': 4, '$': 5}, 5: {'a': 6}}
False
{1: {'a': 1, '$': 2}, 2: {'a': 3}}
True
{1: {'a': 2}, 2: {'b': 3}, 3: {'.': 3, '$': 4}, 4: {'d': 5}, 5: {'e': 6}}
True


In [57]:
bool("delattr")^bool("dd")

False

In [65]:
[4][:-2]

[]